월별 수요량 예측 시각화

In [ ]:
import pandas as pd
import seaborn as sns
#import plotly as plt
import plotly.express as px
import os
import matplotlib.pyplot as plt
import matplotlib as mp
import datetime
import pymysql


In [ ]:
mp.rcParams['font.family'] ='NanumGothic'
mp.rcParams['axes.unicode_minus'] =False

In [ ]:
data = pd.read_csv('list_2.csv', encoding='cp949')

In [ ]:
data['도'].unique()

In [ ]:
jeju_data = data[data['시군구'] == '제주시']
seoguipo_data = data[data['시군구'] == '서귀포시']
susung_data = data[data['시군구'] == '수성구']
jingu_data = data[data['시군구'] == '부산진구']
chungju_data = data[data['시군구'] == '청주시']

In [ ]:
def transtype(data):
    columns_to_drop = ['연', '월', '도', '시군구']
    data['연'] = data['연'].astype(str)
    data['월'] =data['월'].astype(str)
    data['연월'] = data['연'] + '-' + data['월']
    data['지역'] = data['도'] + ' ' + data['시군구']
    data.drop(columns= columns_to_drop,inplace = True)
    data = data[['연월','지역','월별 수요량']]
    data['연월'] = pd.to_datetime(data['연월'])
    return data 

In [ ]:
transtype(chungju_data)

In [ ]:
jeju_data['연월'] = jeju_data['연'] + '-' + jeju_data['월']

In [ ]:
jeju_data['지역'] = jeju_data['도'] +' ' +jeju_data['시군구']

In [ ]:
columns_to_drop = ['연', '월', '도', '시군구']

In [ ]:
jeju_data = jeju_data[['연월','지역','월별 수요량']]

In [ ]:
jeju_data['연월'] = pd.to_datetime(jeju_data['연월'])
jeju_data['연월'] = jeju_data['연월'].dt.to_period('M')

In [ ]:
transtype(jeju_data)
transtype(seoguipo_data)
transtype(jingu_data)
transtype(susung_data)
transtype(chungju_data)

In [ ]:
import seaborn as sns
import plotly.graph_objects as go

In [543]:
fig1 = go.Figure()
#(layout=go.Layout(title=go.layout.Title(text="Title 설정하기")

In [544]:
fig1.add_trace(go.Scatter(x= jeju_data['연월'], y=jeju_data['월별 수요량'], mode='lines', name = '제주도 제주시')) 
fig1.add_trace(go.Scatter(x= seoguipo_data['연월'], y=seoguipo_data['월별 수요량'], mode='lines',name = '제주도 서귀포시'))
fig1.add_trace(go.Scatter(x= jingu_data['연월'], y=jingu_data['월별 수요량'], mode='lines',name = '부산 진구')) 
fig1.add_trace(go.Scatter(x= susung_data['연월'], y=susung_data['월별 수요량'], mode='lines',name = '대구 수성구'))
fig1.add_trace(go.Scatter(x= chungju_data['연월'], y=chungju_data['월별 수요량'], mode='lines',name = '충북 청주'))

In [541]:
fig2 = px.line(jeju_data, x = '연월', y = '월별 수요량',title= '제주시 월별 수요량 변화', markers = True)
#fig2.add_annotation(x = '2023-03', y = '484431', text='충전요금 인상 공지')
fig2.add_annotation(
        x='2023-03',
        y='484431',
        xref="x",
        yref="y",
        text="충전요금 인상 공지",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=10,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=-2,
        ay= -60,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )
fig3 = px.line(seoguipo_data, x = '연월', y = '월별 수요량',title= '서귀포시 월별 수요량 변화', markers = True)
fig4 = px.line(jingu_data, x = '연월', y = '월별 수요량',title= '부산 진구 월별 수요량 변화', markers = True)
fig5 = px.line(susung_data, x = '연월', y = '월별 수요량',title= '대구 수성구 월별 수요량 변화', markers = True)
fig6 = px.line(chungju_data, x = '연월', y = '월별 수요량',title= '충북 청주시 월별 수요량 변화', markers = True)

In [542]:
fig2.show()
fig3.show()
fig4.show()
fig5.show()
fig6.show()

2


In [ ]:
conn = pymysql.connect(host='13.125.65.37',port=3306,user='encore4',password='EnCore4!@#',db='encore4')

In [ ]:
sql = 'select * from gasStationInfo'
sql1 = 'select * from popInfo'

In [ ]:
gas_count = pd.read_sql_query(sql,conn)
popInfo = pd.read_sql_query(sql1,conn)

In [ ]:
gas_count.drop(columns= ['station_name','address','company','phone_number','lag','lat'], axis = 1, inplace = True)

In [ ]:
gas_count['sigungu'].drop_duplicates().count()

In [ ]:
popInfo.drop(columns=['address','pop_per_family'],inplace = True)

In [ ]:
popInfo

In [ ]:
grouped_counts = gas_count.groupby('sigungu').count()

In [ ]:
grouped_counts = grouped_counts.reset_index()

In [ ]:
grouped_counts

In [ ]:
grouped_counts.rename(columns= {'sido' : 'count'})

In [ ]:
popInfo = popInfo[popInfo['sigungu'] != '합계']

In [ ]:
popInfo = popInfo.sort_values('sigungu')

In [ ]:
gas_count = gas_count.sort_values('sigungu')

In [ ]:
grouped_counts = grouped_counts.sort_values('sigungu')

In [ ]:
grouped_counts = grouped_counts.rename(columns = {'sido' : 'count'})

In [ ]:
grouped_counts

In [ ]:
popInfo.drop('sigungu',axis = 1 , inplace = True)

In [ ]:
popInfo

In [ ]:
print(grouped_counts)

In [ ]:
print(popInfo)

In [ ]:
merged_df = pd.merge(grouped_counts, popInfo, on='sigungu', how='left')
merged_df.rename(columns={'count': 'gas_count'}, inplace=True)

In [ ]:
merged_df = merged_df.reset_index()

In [ ]:
merge_df = merged_df[['sido','sigungu','total_pop','total_family','gas_count']]

In [ ]:
merge_df = merge_df.fillna(0)

In [ ]:
merge_df['total_pop'] = merge_df['total_pop'].astype(int)
merge_df['total_family'] = merge_df['total_family'].astype(int)

In [ ]:
merge_df['pop/gas'] = merge_df['total_pop'] / merge_df['gas_count']
merge_df['family/gas'] = merge_df['total_family'] / merge_df['gas_count']

In [ ]:
merge_df['pop/gas'] = merge_df['pop/gas'].astype(int)
merge_df['family/gas'] = merge_df['family/gas'].astype(int)

In [ ]:
merge_df.sort_values('pop/gas', ascending= False)

In [ ]:
pop_gas = merge_df

In [ ]:
pop_gas = merge_df[['sido','']]

In [ ]:
sql = '''
SELECT sido, sigungu FROM gasStationInfo
'''
#chargeStationInfo_df = pd.read_sql_query(sql, con=conn)
#chargeStationInfo_df['sido'].value_counts().sort_values(ascending=False)


In [ ]:
gasStationInfo_df = pd.read_sql_query(sql,conn)

In [ ]:
chargeStationInfo_df['sido'].value_counts().sort_values(ascending=False)

In [ ]:
chargeStationInfo_df.groupby('sido').count()
chargeStationInfo = chargeStationInfo_df.groupby('sido').count()

In [ ]:
chargeStationInfo = chargeStationInfo.reset_index()

In [ ]:
chargeStationInfo = chargeStationInfo.rename(columns = {'count' : 'sigungu'})


In [ ]:
fig = px.bar(data_frame = chargeStationInfo, x = 'sido', y = 'sigungu' )
fig.show()

In [ ]:
sql = '''
SELECT sido, sigungu FROM gasStationInfo
'''

In [ ]:
gasStationInfo_df = pd.read_sql_query(sql,conn)

In [ ]:
gasStationInfo_df.groupby('sido').count()
gasStationInfo = gasStationInfo_df.groupby('sido').count()

In [ ]:
gasStationInfo = gasStationInfo.reset_index() 

In [ ]:
chargeStationInfo

In [ ]:
gasStationInfo = chargeStationInfo.reset_index()

In [ ]:
fig1 = px.bar(data_frame = gasStationInfo, x = 'sido', y = 'sigungu' )
fig1.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
 

In [ ]:
fig3 = go.Figure(data=[gasStationInfo,chargeStationInfo ], layout=layout)
fig3.show()

In [ ]:
data1 = go.Bar(x = gasStationInfo['sido'], y = gasStationInfo['sigungu'], name='화석연료')
data2 = go.Bar(x = chargeStationInfo['sido'], y = chargeStationInfo['sigungu'], name='전기')

In [ ]:
layout = go.Layout(title='충전소 수 비교')

In [ ]:
layout = go.Layout(title='충전소 수 비교')
fig5 = go.Figure(data = [data1, data2], layout=layout)
fig5.show()

In [ ]:
sql = '''
SELECT sido, count FROM electricCarRegitrectionCount
WHERE year=2023'''

In [ ]:
ev_df = pd.read_sql_query(sql,conn)

In [ ]:
ev_df

In [ ]:
sql = '''
SELECT sido, sigungu, count FROM carRegitrectionCount
WHERE year=2023 AND month=12
'''

In [ ]:
gas_df = pd.read_sql_query(sql,conn)

In [ ]:
gas_df.drop('sigungu',axis = 1, inplace = True)

In [ ]:
gas_df_group = gas_df.groupby('sido').sum().reset_index()

In [ ]:
gas_df_group

In [ ]:
data3 = go.Bar(x = ev_df['sido'], y = ev_df['count'], name='전기차')
data4 = go.Bar(x = gas_df_group['sido'], y = gas_df_group['count'], name='화석연료')

In [ ]:
layout = go.Layout(title='차량 수 비교')
fig6 = go.Figure(data = [data3, data4], layout=layout)
fig6.show()

In [ ]:
new_ev = ev_df.sort_values('count',ascending = False).head(5)

In [ ]:
new_gas = gas_df_group.sort_values('count', ascending = False).head(5)

In [ ]:
fig7 = px.bar(data_frame = new_ev, x = 'sido', y = 'count' ,title = '전기차 대수')
fig7.show()

In [ ]:
fig8 = px.bar(data_frame = new_gas, x = 'sido', y = 'count', title = '화석연료 대수')
fig8.show()